In [1]:
#!nvidia-smi

In [2]:
!pip install ultralytics motrackers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
from ultralytics import YOLO
from motrackers import CentroidTracker
import cv2
import numpy as np
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
main_path = "/content/gdrive/MyDrive/Chicken Health and Behavior Detection"
destination_path = '/content'

In [6]:
!cp "$main_path/models/object_detection/model_yolov11s.zip" "$destination_path"

In [7]:
!unzip -q "$destination_path/model_yolov11s.zip" -d "$destination_path/model"

In [8]:
model = YOLO(f'{destination_path}/model/yolov11s.pt')

In [9]:
video_path = f'{main_path}/data/processed/video_test.mp4'
output_path = f'{destination_path}/output_video_test_tw_distance.avi'

In [10]:
# Open video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

# Tracker
tracker = CentroidTracker(max_lost=5, tracker_output_format='mot_challenge')
track_data = {}  # {id: {'centroid': (x, y), 'total_distance': float}}
frame_count = 0

In [11]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Run YOLOv11 inference
    results = model(frame)

    # Get predictions
    detections = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()       # Bounding box coordinates
        confidences = result.boxes.conf.cpu().numpy() # Confidence scores
        class_ids = result.boxes.cls.cpu().numpy()    # Class IDs

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box.astype(int)
            w = x2 - x1
            h = y2 - y1
            detections.append([x1, y1, w, h])

    # Update tracker
    tracks = tracker.update(np.array(detections), confidences, class_ids)

    # Draw tracks and calculate distances
    for trk in tracks:
        track_id = int(trk[1])
        xmin, ymin, w, h = map(int, trk[2:6])
        cx = xmin + w // 2
        cy = ymin + h // 2

        # Initialize data if new ID
        if track_id not in track_data:
            track_data[track_id] = {
                'centroid': (cx, cy),
                'total_distance': 0.0
            }

        # Calculate distance from previous position
        prev_cx, prev_cy = track_data[track_id]['centroid']
        dx = cx - prev_cx
        dy = cy - prev_cy
        dist = np.sqrt(dx**2 + dy**2)

        # Filter out unrealistic jumps (optional)
        if dist > 30:
            dist = 0

        # Update total distance and centroid
        track_data[track_id]['total_distance'] += dist
        track_data[track_id]['centroid'] = (cx, cy)

        # Display ID and TDP
        tdp = round(track_data[track_id]['total_distance'], 1)
        label = f"ID {track_id} | TDP: {tdp}"
        cv2.putText(frame, label, (xmin, ymin - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.rectangle(frame, (xmin, ymin), (xmin + w, ymin + h), (0, 255, 0), 1)
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)

    # Write to output video
    out.write(frame)
    frame_count += 1

    print(f"Processed frame {frame_count} | Time: {time.time() - start_time:.3f}s")


0: 384x640 62 0s, 48.7ms
Speed: 19.5ms preprocess, 48.7ms inference, 410.5ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1 | Time: 6.692s

0: 384x640 61 0s, 17.0ms
Speed: 2.5ms preprocess, 17.0ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2 | Time: 0.064s

0: 384x640 61 0s, 12.2ms
Speed: 2.5ms preprocess, 12.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3 | Time: 0.039s

0: 384x640 60 0s, 11.4ms
Speed: 3.5ms preprocess, 11.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4 | Time: 0.040s

0: 384x640 60 0s, 13.4ms
Speed: 2.1ms preprocess, 13.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5 | Time: 0.040s

0: 384x640 60 0s, 11.3ms
Speed: 2.1ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 6 | Time: 0.038s

0: 384x640 60 0s, 11.3ms
Speed: 2.3ms preprocess, 11.3ms inf

In [12]:
cap.release()
out.release()